In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import matplotlib.pyplot as plt
csv_file = "dataQuestionnaire.csv"
data = pd.read_csv(csv_file)

In [2]:
print(data.shape)

(741, 90)


In [3]:
texte_colonnes = [
    "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)",
    "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)",
    "Pouvez-vous expliquer vos réponses merci ..."
]

for colonne in texte_colonnes:
    nb_non_null = data[colonne].count()
    print(f"Nombre de valeurs non nulles dans la colonne '{colonne}': {nb_non_null}")


Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)': 104
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)': 183
Nombre de valeurs non nulles dans la colonne 'Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)': 185
Nombre de valeurs non nulles dans la colonne 'Pouvez-vous expliquer vos réponses merci ...': 366


In [4]:
negatif = ['pas', 'charge', 'trop', 'peu', 'privé', 'administratif', 'administrative', 'perte', 'administratifs', 'but',
           'décharge', 'surcharge', 'covid', 'dégradé', 'perdre', 'doute', 'arrêter', 'admin', 'burn', 'cdd', 'out', 'refus',
           'empiète', 'fatigue', 'fou', 'cher', 'démotive', 'empêche', 'négatif', 'épuisant', 'adm', 'burnout', 'copinage',
           'décourageant', 'décrocher', 'frein', 'frustrant', 'handicap', 'misère', 'aucune', 'aucun', 'poids', 'sacrifier',
           'smic', 'soucis', 'souffre', 'transport', 'décourage', 'dégradée', 'forcé', 'fuite', 'guerre', 'hypocrisie',
           'insuffisant', 'négatifs', 'souci', 'tue', 'catastrophe', 'décourager', 'déplacement', 'déplacer', 'réforme', 'ni']

neutre = ['temps', 'métier', 'étudiants', 'si', 'carrière', 'chercheurs', 'nombre', 'ans', 'chercheur', 'année', 'post', 'cours',
          'activité', 'mcf', 'poste', 'doctorants', 'pouvoir', 'rapport', 'compte', 'vie', 'passé', 'trouver', 'activités', 'iut',
         'support', 'avant', 'après', 'dossier', 'compris', 'dire', 'fois', 'forte', 'important', 'choix', 'docteur', 'début', 'juste',
         'voir', 'étant', 'gérer', 'fin', 'futur', 'intérêt', 'ufr', 'doctorant', 'dossiers', 'candidats', 'arrêté', 'pédagogie', 'cnrs',
          'financier', 'aspect', 'cnu', 'suivi', 'étudiant', 'ingénieur', 'métiers', 'offre', 'ingénieurs', 'sup', 'état', 'assurer',
          'directeur', 'doctorat', 'recruter', 'supérieur', 'société', 'prof', 'avis', 'effectif', 'effectifs', 'statut', 'candidat',
          'chef', 'santé', 'structure', 'thésards', 'tâche', 'impact', 'iuf', 'ministère', 'perspective', 'stage', 'stratégie', 'sit',
          'starynkevitch', 'refpersys']

positif = ['recherche', 'sur', 'sans', 'ripec', 'docteurs', 'attirant', 'étranger', 'anr', 'hdr', 'attractif', 'attractivité',
           'ripec3', 'inria', 'cadre', 'ins2i', 'motivés', 'avantage', ' apprécie', 'attirer', 'attractifs', 'gratifiant', 'motivant',
           'réussi', 'succès', 'suffisant', 'bénéficier', 'cdi', 'positifs', 'ras', 'réussir', 'réussite', 'rêve', 'bénéficie',
           'motivé', 'mérite', 'positif', 'rechercher']

#mots_pas_sur = ['ripec', 'anr', 'hdr', 'cnrs', 'cnu', 'ripec3', 'sit', 'inria', 'starynkevitch', 'ins2i', 'refpersys']

In [5]:
"""
negatif = ['trop', 'peu', 'perte',
           'décharge', 'surcharge', 'covid', 'dégradé', 'perdre', 'doute', 'burn', 'out', 'refus',
           'empiète', 'fatigue', 'fou', 'cher', 'démotive', 'empêche', 'négatif', 'épuisant', 'burnout', 'copinage',
           'décourageant', 'décrocher', 'frein', 'handicap', 'misère', 'sacrifier',
           'souffre', 'décourage', 'dégradée', 'forcé', 'fuite', 'guerre', 'hypocrisie',
           'insuffisant', 'négatifs', 'souci', 'tue', 'catastrophe', 'décourager', 'ni']

positif = [ 'sur', 'attirant',  'attractif', 'attractivité',
            'motivés', 'avantage', 'apprécie', 'attirer', 'attractifs', 'gratifiant', 'motivant',
           'réussi', 'succès', 'suffisant', 'bénéficier', 'positifs', 'réussir', 'réussite', 'rêve', 'bénéficie',
           'motivé', 'mérite', 'positif']
"""

"\nnegatif = ['trop', 'peu', 'perte',\n           'décharge', 'surcharge', 'covid', 'dégradé', 'perdre', 'doute', 'burn', 'out', 'refus',\n           'empiète', 'fatigue', 'fou', 'cher', 'démotive', 'empêche', 'négatif', 'épuisant', 'burnout', 'copinage',\n           'décourageant', 'décrocher', 'frein', 'handicap', 'misère', 'sacrifier',\n           'souffre', 'décourage', 'dégradée', 'forcé', 'fuite', 'guerre', 'hypocrisie',\n           'insuffisant', 'négatifs', 'souci', 'tue', 'catastrophe', 'décourager', 'ni']\n\npositif = [ 'sur', 'attirant',  'attractif', 'attractivité',\n            'motivés', 'avantage', 'apprécie', 'attirer', 'attractifs', 'gratifiant', 'motivant',\n           'réussi', 'succès', 'suffisant', 'bénéficier', 'positifs', 'réussir', 'réussite', 'rêve', 'bénéficie',\n           'motivé', 'mérite', 'positif']\n"

In [6]:
personnel = data[["SID", texte_colonnes[0]]].dropna()
print("Premières lignes de personnel après suppression des valeurs nulles :\n", personnel.head())

enseignement = data[["SID", texte_colonnes[1]]].dropna()
print("Premières lignes de enseignement après suppression des valeurs nulles :\n", enseignement.head())

recherche = data[["SID", texte_colonnes[2]]].dropna()
print("Premières lignes de recherche après suppression des valeurs nulles :\n", recherche.head())

merci = data[["SID", texte_colonnes[3]]].dropna()
print("Premières lignes de merci après suppression des valeurs nulles :\n", merci.head())

Premières lignes de personnel après suppression des valeurs nulles :
          SID  \
1   15896547   
10  15897993   
14  15899015   
15  15899810   
20  15901382   

   Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  
1   Il n'y a plus d'UFR à l'Université de Bordeaux...                             
10                   Temps moyen hors période bouchon                             
14  Temps moyen de déplacement selon les sites, pe...                             
15  Je suis Responsable de M2, mon travail a une f...                             
20  Pas de distance en temps pour transiter du lie...                             
Premières lignes de enseignement après suppression des valeurs nulles :
          SID  \
4   15896653   
10  15897993   
13  15898762   
14  15899015   
15  15899810   

   Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)  
4   L'enseignement est de plus en plus difficile. ...        

In [7]:
print(personnel.shape)
print(enseignement.shape)
print(recherche.shape)
print(merci.shape)

(104, 2)
(183, 2)
(185, 2)
(366, 2)




*   Calculer la note pour chaque commentaire
*   Afficher le nombre d'occurrences de chaque note
*   Afficher les premières lignes du DataFrame
*   Compter le nombre de notes positives, neutres et négatives





In [8]:
def compter_mots_positifs_negatifs(commentaire, liste_positif, liste_negatif):
    compteur_positif = 0
    compteur_negatif = 0

    mots = commentaire.lower().split()
    for mot in mots:
        if mot in liste_positif:
            compteur_positif += 1
        elif mot in liste_negatif:
            compteur_negatif += 1

    return compteur_positif - compteur_negatif

personnel["Note personnel"] = personnel[texte_colonnes[0]].apply(lambda x: compter_mots_positifs_negatifs(x, positif, negatif))
print(personnel["Note personnel"].value_counts())
personnel.head()

 0    55
 1    21
-1    18
-2     5
 2     3
 3     1
-3     1
Name: Note personnel, dtype: int64


,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Note personnel
1,15896547,Il n'y a plus d'UFR à l'Université de Bordeaux...,0
10,15897993,Temps moyen hors période bouchon,0
14,15899015,"Temps moyen de déplacement selon les sites, pe...",-1
15,15899810,"Je suis Responsable de M2, mon travail a une f...",0
20,15901382,Pas de distance en temps pour transiter du lie...,1


In [9]:
nb_positif = (personnel["Note personnel"] > 0).sum()
nb_neutre = (personnel["Note personnel"] == 0).sum()
nb_negatif = (personnel["Note personnel"] < 0).sum()

print("Pour Note personnel :")
print("Nombre de notes positives :", nb_positif)
print("Nombre de notes neutres :", nb_neutre)
print("Nombre de notes négatives :", nb_negatif)

Pour Note personnel :
Nombre de notes positives : 25
Nombre de notes neutres : 55
Nombre de notes négatives : 24


In [10]:
enseignement["Note enseignement"] = enseignement[texte_colonnes[1]].apply(lambda x: compter_mots_positifs_negatifs(x, positif, negatif))
print(enseignement["Note enseignement"].value_counts())
enseignement.head()

 0     67
-1     49
-2     29
 1     15
-3      8
 2      6
-4      4
-5      2
 3      2
-14     1
Name: Note enseignement, dtype: int64


,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement
4,15896653,L'enseignement est de plus en plus difficile. ...,-2
10,15897993,"A ma connaissance, mon enseignement n'est pas ...",-1
13,15898762,"Attention, pour être reconnu, l'investissement...",0
14,15899015,"dernière question, ""S'il y a"" et non pas ""S'i...",-1
15,15899810,Quand un jeune me demande si devenir prof de f...,-2


In [11]:
nb_positif = (enseignement["Note enseignement"] > 0).sum()
nb_neutre = (enseignement["Note enseignement"] == 0).sum()
nb_negatif = (enseignement["Note enseignement"] < 0).sum()

print("Pour note enseignement :")
print("Nombre de notes positives :", nb_positif)
print("Nombre de notes neutres :", nb_neutre)
print("Nombre de notes négatives :", nb_negatif)

Pour note enseignement :
Nombre de notes positives : 23
Nombre de notes neutres : 67
Nombre de notes négatives : 93


In [12]:
recherche["Note recherche"] = recherche[texte_colonnes[2]].apply(lambda x: compter_mots_positifs_negatifs(x, positif, negatif))
print(recherche["Note recherche"].value_counts())
recherche.head()

 0    60
 1    39
-1    37
-2    17
 2    10
 3     7
-3     6
-6     2
 4     2
-5     2
 5     1
-4     1
 6     1
Name: Note recherche, dtype: int64


,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Note recherche
4,15896653,Je fais ma recherche quasiment toute seule dep...,2
5,15896801,Je suis dans une équipe Inria. Cela offre des ...,1
10,15897993,Auparavant au CEA (R&D)\r\nMigration Universit...,-1
12,15898367,Les conditions d'exercices sont de plus en plu...,2
13,15898762,Questionnement similaire à celui de Romain Cou...,0


In [13]:
nb_positif = (recherche["Note recherche"] > 0).sum()
nb_neutre = (recherche["Note recherche"] == 0).sum()
nb_negatif = (recherche["Note recherche"] < 0).sum()

print("Pour note recherche :")
print("Nombre de notes positives :", nb_positif)
print("Nombre de notes neutres :", nb_neutre)
print("Nombre de notes négatives :", nb_negatif)

Pour note recherche :
Nombre de notes positives : 60
Nombre de notes neutres : 60
Nombre de notes négatives : 65


In [14]:
merci["Note merci"] = merci[texte_colonnes[3]].apply(lambda x: compter_mots_positifs_negatifs(x, positif, negatif))
print(merci["Note merci"].value_counts())
merci.head()

 0    114
-1     71
-2     55
 1     41
-3     25
 2     22
-5     10
-4     10
 3      9
-6      5
-8      1
 4      1
 6      1
-7      1
Name: Note merci, dtype: int64


,SID,Pouvez-vous expliquer vos réponses merci ...,Note merci
2,15896551,Les conditions de travail dépendent beaucoup d...,0
4,15896653,"Je n'ai aucun espoir de passer un jour PU, ni ...",-2
5,15896801,On garde l'espoir.,0
7,15897130,"Aujourd’hui, l’informatique offre une multitud...",1
10,15897993,Profession en perte de considération dans la s...,-2


In [15]:
nb_positif = (merci["Note merci"] > 0).sum()
nb_neutre = (merci["Note merci"] == 0).sum()
nb_negatif = (merci["Note merci"] < 0).sum()

print("Pour note merci :")
print("Nombre de notes positives :", nb_positif)
print("Nombre de notes neutres :", nb_neutre)
print("Nombre de notes négatives :", nb_negatif)

Pour note merci :
Nombre de notes positives : 74
Nombre de notes neutres : 114
Nombre de notes négatives : 178


In [16]:
total = data[["SID"]].copy()
total["Totalite des commentaires"] = data[texte_colonnes].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
total = total.dropna(subset=["Totalite des commentaires"])

total["Note total"] = total["Totalite des commentaires"].apply(lambda x: compter_mots_positifs_negatifs(x, positif, negatif))
print(total["Note total"].value_counts())
total.head()

 0     428
-1      77
-2      60
 1      55
-3      29
 2      24
-5      18
-4      13
-6       7
 3       7
-7       5
-8       5
 4       4
 6       3
 5       2
-11      2
-18      1
-9       1
Name: Note total, dtype: int64


,SID,Totalite des commentaires,Note total
0,15792852,,0
1,15896547,Il n'y a plus d'UFR à l'Université de Bordeaux...,0
2,15896551,Les conditions de travail dépendent beaucoup d...,0
3,15896563,,0
4,15896653,L'enseignement est de plus en plus difficile. ...,-2


In [18]:
nb_positif = (total["Note total"] > 0).sum()
nb_neutre = (total["Note total"] == 0).sum()
nb_negatif = (total["Note total"] < 0).sum()

print("Pour note total :")
print("Nombre de notes positives :", nb_positif)
print("Nombre de notes neutres :", nb_neutre)
print("Nombre de notes négatives :", nb_negatif)

Pour note total :
Nombre de notes positives : 95
Nombre de notes neutres : 428
Nombre de notes négatives : 218


In [19]:
merged_df = pd.merge(personnel, enseignement, on='SID', how='outer')
merged_df = pd.merge(merged_df, recherche, on='SID', how='outer')
merged_df = pd.merge(merged_df, merci, on='SID', how='outer')
merged_df = pd.merge(merged_df, total, on='SID', how='outer')

merged_df.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Note personnel,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Note enseignement,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Note recherche,Pouvez-vous expliquer vos réponses merci ...,Note merci,Totalite des commentaires,Note total
0,15896547,Il n'y a plus d'UFR à l'Université de Bordeaux...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Il n'y a plus d'UFR à l'Université de Bordeaux...,0
1,15897993,Temps moyen hors période bouchon,0.0,"A ma connaissance, mon enseignement n'est pas ...",-1.0,Auparavant au CEA (R&D)\r\nMigration Universit...,-1.0,Profession en perte de considération dans la s...,-2.0,Temps moyen hors période bouchon A ma connaiss...,-4
2,15899015,"Temps moyen de déplacement selon les sites, pe...",-1.0,"dernière question, ""S'il y a"" et non pas ""S'i...",-1.0,NaN,NaN,On peut faire notre recherche comme on le souh...,-2.0,"Temps moyen de déplacement selon les sites, pe...",-4
3,15899810,"Je suis Responsable de M2, mon travail a une f...",0.0,Quand un jeune me demande si devenir prof de f...,-2.0,J'ai arrêté pour les raisons indiquées dans mo...,0.0,Non sans hésitation aux trois questions. Voir ...,1.0,"Je suis Responsable de M2, mon travail a une f...",-1
4,15901382,Pas de distance en temps pour transiter du lie...,1.0,NaN,NaN,J'aime toujours faire de la recherche mais j'a...,1.0,Métier intéressant car très varié et avec des ...,0.0,Pas de distance en temps pour transiter du lie...,2


In [20]:
merged_df.to_csv("notation_manuelle.csv", index=False)